#Bibliotecas

In [ ]:
#Importando o tensorflow
import tensorflow as tf
#Importanto um leitor e prepocessador de imagens para lermos imagens de um diretório
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#Importanto o matplotlib para plotar as imagens
import matplotlib.pyplot as plt
#Importanto o numpy para fazer algumas operações nos vetores das imagens
import numpy as np

#Instanciando um Gerador de Imagens

In [ ]:
diretorio_treinamento = '/content/drive/MyDrive/Datasets/Imagens/cats_and_dogs/train'
diretorio_teste = '/content/drive/MyDrive/Datasets/Imagens/cats_and_dogs/test'

In [ ]:
"""O ImageDataGenerator já irar levar as imagens e 
   padronizar os valores entre 0 e 1"""
data_generator = ImageDataGenerator(rescale=1/255.0)

In [ ]:
#Obtendo as imagens do diretório de dados de acordo com os parâmetros do treinamento
#- 1º parâmetro: diretório com as imagens de treinamento
#- Batch size: tamanho dos conjuntos de imagens que serão submetidas a rede a cada iteração
#- Target size: resolução da imagem 
#- Formato dos labels: podem ser valores inteiros, one_hot_encodding, ou esparsos (nome da pasta)
gerador_treino = data_generator.flow_from_directory(diretorio_treinamento, batch_size=32, target_size=(224,224), class_mode='sparse')

In [ ]:
gerador_teste = data_generator.flow_from_directory(diretorio_teste, batch_size=32, target_size=(224,224), class_mode='sparse')

In [ ]:
len(gerador_treino)

In [ ]:
gerador_treino[[1,2]]

In [ ]:
#Visualizando o conteúdo de um elemento do gerador
gerador_treino[0]

In [ ]:
#Imagens do primeiro batch
gerador_treino[0][0].shape

In [ ]:
#classes do primeiro batch
gerador_treino[0][1].shape

In [ ]:
#Plotando uma imagem do conjunto de treino
plt.imshow(gerador_treino[0][0][0])

#Obtendo um modelo de rede neural (MobileNet)

In [ ]:
#Include_top = False significa que não estamos importando a parte final da redes (camadas densas)
model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3), include_top=False)

In [ ]:
#Criando camadas para a parte final da rede e linkando com a ultima camada do modelo base
average_layer = tf.keras.layers.GlobalAveragePooling2D()(model.output)

In [ ]:
#Criando camadas para a parte final da rede e linkando com a ultima camada do modelo base
hidden_dense = tf.keras.layers.Dense(128,activation='relu')(average_layer)

In [ ]:
#Criando a camada de predição
preds = tf.keras.layers.Dense(1,activation='sigmoid')(hidden_dense)

In [ ]:
#Juntando a base com as partes criadas
model_final = tf.keras.models.Model(inputs=model.input,outputs=preds)

#Sumário do Modelo

In [ ]:
model_final.summary()

# Compilando o Modelo

In [ ]:
model_final.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#Treiando o Modelo

In [ ]:
model_final.fit(gerador_treino, epochs=10)

# Avaliando o Modelo

In [ ]:
loss, accuracy = model_final.evaluate(gerador_teste)

In [ ]:
loss

In [ ]:
accuracy

# Vendo o resultado na prática

In [ ]:
plt.imshow(gerador_teste[2][0][0])

In [ ]:
teste_imagem = gerador_teste[0][0][0].reshape(1,224,224,3)

In [ ]:
# A priori, como é uma classificação binária, podemos definir um limiar < 0.5 = gato e >= 0.5 cachoro
model_final.predict(teste_imagem)

In [ ]:
plt.imshow(gerador_teste[9][0][0])

In [ ]:
teste_imagem2 = gerador_teste[9][0][0].reshape(1,224,224,3)

In [ ]:
model_final.predict(teste_imagem2)